In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import LocalOutlierFactor

In [57]:
vectors = pd.read_csv("data/extracted_n26_tsv_vecs.tsv", delimiter='\t|,', header=None, engine='python')
vectors = vectors.drop(vectors.columns[0], axis=1)
metadata = pd.read_csv("data/extracted_n26_tsv_metadata.tsv", delimiter='\t')

vec_meta = vectors.join(metadata)
train_data = vec_meta[vec_meta['FAQ_id'] <= 135] # only train on first 130 classes
train_data = train_data.drop(['FAQ_id', 'locale', 'market', 'question'], axis=1)
new_data = vec_meta[vec_meta['FAQ_id'] > 135]
new_data = new_data.drop(['FAQ_id', 'locale', 'market', 'question'], axis=1)

In [27]:
# Local outlier detection
lof = LocalOutlierFactor(n_neighbors=10, contamination='auto')
train_data['lof'] = lof.fit_predict(train_data)

# Getting the negative LOF score
train_data['negative_outlier_factor'] = lof.negative_outlier_factor_
# print(train_data.head)

outliers = train_data[train_data['lof'] == -1]
print(outliers.head)

<bound method NDFrame.head of              1         2         3         4         5         6         7  \
17    0.032272  0.037160  0.023964  0.079723 -0.013657  0.043202  0.056083   
45   -0.025578 -0.041987 -0.063839 -0.051133  0.028106 -0.061051  0.022543   
46   -0.032095 -0.055612 -0.071654 -0.034519  0.032207 -0.062536  0.028935   
71   -0.088064  0.068836 -0.067431  0.019893 -0.024917 -0.080645  0.000643   
157  -0.020392 -0.000453  0.009813 -0.069363  0.003863 -0.098193  0.013508   
182   0.003621  0.016248 -0.056824  0.067728 -0.035488 -0.064897  0.016880   
189   0.003621  0.016248 -0.056824  0.067728 -0.035488 -0.064897  0.016880   
288  -0.019817  0.045492 -0.074934  0.104480 -0.011956 -0.082876  0.011879   
344  -0.030202  0.040071  0.007633  0.006708 -0.007241 -0.024703 -0.076389   
436   0.067351  0.034778 -0.077544  0.120391  0.069748 -0.042529 -0.004198   
447   0.067351  0.034778 -0.077544  0.120391  0.069748 -0.042529 -0.004198   
581   0.022559  0.022454 -0.064656

In [58]:
# Novelty detection
lof = LocalOutlierFactor(n_neighbors=15, contamination='auto', novelty=True)
lof.fit(train_data)

# new_data['novelty'] = lof.predict(new_data)
# print(new_data[new_data['novelty'] == 1])

y_pred_test = lof.predict(new_data)
n_correct_test = y_pred_test[y_pred_test == 1].size
print(n_correct_test)
print("Accuracy: " + str(n_correct_test / new_data.shape[0]))

20
Accuracy: 1.0
